In [160]:
import pandas as pd

df_2 = pd.read_csv('IMDB Dataset.csv', sep=',')

# reduzindo df_2
df_2.drop(range(49000), inplace=True)
df_2.reset_index(drop=True, inplace=True)

# limpando o df
df_2['review'] = df_2['review'].str.replace('<br>', '', regex=True)
df_2['review'] = df_2['review'].str.replace('[^\w\s]', '',regex=True )
df_2['review'] = df_2['review'].str.lower()

train_size = int(0.5 * len(df_2))

train_review = df_2['review'][:train_size]
train_sentiment = df_2['sentiment'][:train_size]

test_review = df_2['review'][train_size:]
test_sentiment = df_2['sentiment'][train_size:]

# df de treinamento
df_train = pd.DataFrame()
df_train['review'] = train_review
df_train['sentiment'] = train_sentiment

# df de teste
df_teste = pd.DataFrame()
df_teste['review'] = test_review
df_teste['sentiment'] = test_sentiment

df_2.head()

,review,sentiment
0,i havent laughed this hard at a movie in a lon...,positive
1,well this movie was probobly one of the funnie...,positive
2,plot none script a string of cliches acting no...,negative
3,i am dumbfounded that i actually sat and watch...,negative
4,there is a certain genius behind this movie i ...,positive


In [161]:
df_positive = df_train[df_train['sentiment'] == 'positive']
df_negative = df_train[df_train['sentiment'] == 'negative']

# registrando todas as palavras do df
dic_palavras = {}
for review in df_train['review']:
    palavras_unicas = set(review.split())
    for palavra in palavras_unicas:
        dic_palavras[palavra] = 0.1

for palavra in dic_palavras:
    for review in df_train['review']:
        lista_palavras = review.split()
        if palavra in lista_palavras:
            dic_palavras[palavra]+= 1



In [162]:
# registrando as palavras que compõem os reviews positivos 
dic_palavras_positivas = {}
for palavra in dic_palavras:
    dic_palavras_positivas[palavra] = 0.1
for review in df_positive['review']:
    lista_palavras_pos = set(review.split())
    for palavra in lista_palavras_pos:
        dic_palavras_positivas[palavra]+= 1

# registrando as palavras que compõem os reviews negativos 
dic_palavras_negativas = {}
for palavra in dic_palavras:
    dic_palavras_negativas[palavra] = 0.1
for review in df_negative['review']:
    lista_palavras_neg = set(review.split())
    for palavra in lista_palavras_neg:
        dic_palavras_negativas[palavra]+= 1

In [163]:
# num de documentos na classe positiva 
num_doc_positiva = len(df_positive)

# num de documentos na classe negativa 
num_doc_negativa = len(df_negative)

#probabilidade review positiva dado df train todo
p_train_positiva = len(df_positive)/len(df_train)

#probabilidade review negativa dado df train todo
p_train_negativa = len(df_negative)/len(df_train)


In [164]:
# p(W)probabilidade de documentos que contém palavra / # total de documentos
prob_palavras = {}
for palavra in dic_palavras:
    prob_palavras[palavra] = dic_palavras[palavra]/len(df_train)

# p(W|C.pos)documentos da classe positiva que contém palavra / # documentos da classe C
prob_palavras_positivas = {}
for palavra in dic_palavras_positivas:
    prob_palavras_positivas[palavra] = dic_palavras_positivas[palavra]/len(df_positive)

# p(W|C.neg)documentos da classe negativa que contém palavra / # documentos da classe C
prob_palavras_negativa = {}
for palavra in dic_palavras_negativas:
    prob_palavras_negativa[palavra] = dic_palavras_negativas[palavra]/len(df_negative)

# p(W negado)
prob_palavras_n = {}
for palavra in dic_palavras:
    prob_palavras_n[palavra] = 1 - prob_palavras[palavra]

# p (W negada| C.pos)
prob_n_palavra_pos = {}
for palavra in dic_palavras_positivas:
    prob_n_palavra_pos[palavra] = 1 - prob_palavras_positivas[palavra]

# p (W negada| C.neg)
prob_n_palavra_neg = {}
for palavra in dic_palavras_negativas:
    prob_n_palavra_neg[palavra] = 1 - prob_palavras_negativa[palavra]

In [165]:


import math

def bayes(df, prob_palavras_positivas, prob_palavras_negativas, dic_palavras, prob_palavras, p_positiva, p_negativa, prob_palavras_n ,prob_n_palavra_pos, prob_n_palavra_neg ):
    resultado = []

    log_p_positiva = math.log(p_positiva)
    log_p_negativa = math.log(p_negativa)
    
    for index, row in df.iterrows():
        review = row['review']
        palavras_review = set(review.split())

        log_prob_pos = log_p_positiva
        log_prob_neg = log_p_negativa

        for palavra in palavras_review:
            # desconsiderar as palavras que não estão no dicionário de modelo
            if palavra not in dic_palavras:
                continue
            if palavra in prob_palavras_positivas:
                log_prob_pos += math.log(prob_palavras_positivas[palavra]) - math.log(prob_palavras[palavra])
            else:
            # probabilidade da palavra ser representante de um review positiva mesmo não estando no dicionario de palavras presentes em reviews positivos 
                log_prob_pos += math.log(prob_n_palavra_pos[palavra]) - math.log(prob_palavras_n[palavra])

            if palavra in prob_palavras_negativas:
                log_prob_neg += math.log(prob_palavras_negativas[palavra]) - math.log(prob_palavras[palavra])
            else:
                log_prob_neg += math.log(prob_n_palavra_neg[palavra]) - math.log(prob_palavras_n[palavra])
  
        log_prob_pos += math.log(p_positiva)
        log_prob_neg += math.log(p_negativa)


        if log_prob_pos > log_prob_neg:
            resultado.append('positive')
        else:
            resultado.append('negative')
            
    return resultado

In [166]:
bayes = bayes(df_teste, prob_palavras_positivas, prob_palavras_negativa, dic_palavras, prob_palavras, p_train_positiva, p_train_negativa, prob_palavras_n ,prob_n_palavra_pos, prob_n_palavra_neg )

acertos = 0

real_sentiment = []
for index, row in df_teste.iterrows():
    sentiment = row['sentiment']
    real_sentiment.append(sentiment)


for i in range(len(bayes)):
    if bayes[i] == real_sentiment[i]:
        acertos += 1


# accuracy = % acertos/ n total de observações vezes 100 para ficar em porcentagem
correct_counts = (acertos/len(df_teste))*100
print(correct_counts)


75.4
